In [97]:
from io import BytesIO

import pandas as pd
import subprocess


In [98]:
scp_return = subprocess.run(["scp", "-i", "~/.ssh/isucon-takanokata.pem", "ubuntu@18.177.155.1:/var/log/nginx/access.log",  "/dev/stdout",], capture_output=True)

In [99]:
access_log = scp_return.stdout.decode("utf-8")

In [100]:
access_log

'- - - [10/Jul/2022:10:16:02 +0000] "POST /initialize HTTP/1.1" 200 31 "-" "benchmarker/isucon9-qualify" 9.347\n- - - [10/Jul/2022:10:16:02 +0000] "GET /static/js/2.ff6e1067.chunk.js HTTP/1.1" 200 508459 "-" "benchmarker/isucon9-qualify" 0.002\n- - - [10/Jul/2022:10:16:02 +0000] "GET /static/js/runtime~main.a8a9905a.js HTTP/1.1" 200 1502 "-" "benchmarker/isucon9-qualify" 0.001\n- - - [10/Jul/2022:10:16:02 +0000] "GET /static/js/main.babc3d4d.chunk.js HTTP/1.1" 200 90365 "-" "benchmarker/isucon9-qualify" 0.001\n- - - [10/Jul/2022:10:16:02 +0000] "GET /static/css/main.19393e92.chunk.css HTTP/1.1" 200 994 "-" "benchmarker/isucon9-qualify" 0.000\n- - - [10/Jul/2022:10:16:02 +0000] "POST /login HTTP/1.1" 200 103 "-" "benchmarker/isucon9-qualify" 0.199\n- - - [10/Jul/2022:10:16:02 +0000] "POST /login HTTP/1.1" 200 100 "-" "benchmarker/isucon9-qualify" 0.228\n- - - [10/Jul/2022:10:16:02 +0000] "POST /login HTTP/1.1" 200 91 "-" "benchmarker/isucon9-qualify" 0.262\n- - - [10/Jul/2022:10:16:02 +

In [101]:
df = pd.read_csv(BytesIO(scp_return.stdout),
              sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
              engine='python',
              usecols=[0, 3, 4, 5, 6, 7, 8, 9],
              names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent', "request_time"],
              na_values='-',
              header=None
                )

In [102]:
df

,ip,time,request,status,size,referer,user_agent,request_time
0,NaN,[10/Jul/2022:10:16:02 +0000],"""POST /initialize HTTP/1.1""",200,31,"""-""","""benchmarker/isucon9-qualify""",9.347
1,NaN,[10/Jul/2022:10:16:02 +0000],"""GET /static/js/2.ff6e1067.chunk.js HTTP/1.1""",200,508459,"""-""","""benchmarker/isucon9-qualify""",0.002
2,NaN,[10/Jul/2022:10:16:02 +0000],"""GET /static/js/runtime~main.a8a9905a.js HTTP/...",200,1502,"""-""","""benchmarker/isucon9-qualify""",0.001
3,NaN,[10/Jul/2022:10:16:02 +0000],"""GET /static/js/main.babc3d4d.chunk.js HTTP/1.1""",200,90365,"""-""","""benchmarker/isucon9-qualify""",0.001
4,NaN,[10/Jul/2022:10:16:02 +0000],"""GET /static/css/main.19393e92.chunk.css HTTP/...",200,994,"""-""","""benchmarker/isucon9-qualify""",0.000
...,...,...,...,...,...,...,...,...
15591,NaN,[10/Jul/2022:10:27:44 +0000],"""GET /users/transactions.json HTTP/1.1""",499,0,"""-""","""benchmarker/isucon9-qualify""",5.395
15592,NaN,[10/Jul/2022:10:27:44 +0000],"""GET /users/transactions.json?created_at=15655...",499,0,"""-""","""benchmarker/isucon9-qualify""",2.774
15593,NaN,[10/Jul/2022:10:27:44 +0000],"""GET /users/transactions.json HTTP/1.1""",499,0,"""-""","""benchmarker/isucon9-qualify""",3.139
15594,NaN,[10/Jul/2022:10:27:44 +0000],"""GET /users/transactions.json HTTP/1.1""",499,0,"""-""","""benchmarker/isucon9-qualify""",4.572


In [103]:
def parse_request(request):
    request = request.split(" ")
    method = request[0]
    path = request[1]

    if path.startswith("/users/") and path.endswith(".json"):
        path = "/users/:id.json"
    if path.startswith("/items/") and path.endswith(".json"):
        path = "/items/:id.json"
    if path.startswith("/transactions/") and path.endswith(".json"):
        path = "/transaction/:id.json"
    if path.startswith("/transactions/") and path.endswith(".png"):
        path = "/transaction/:id.png"

    if path.startswith("/new_items.json"):
        path = "/new_items.json"

    if path.startswith("/new_items/"):
        path = "/new_items/:id.json"
    if path.startswith("/users/"):
        path = "/users/:id.json"
    if path.startswith("/upload/"):
        path = "/upload/:id.jpg"

    return path

In [104]:
df["path"] = df["request"].apply(parse_request)

In [105]:
aggregated_df = df[["path", "request_time"]].groupby("path").agg({
    "request_time": ["count", "mean", "std", "sum", "min", "max"]
})
aggregated_df

request_time                      \
                                           count      mean       std   
path                                                                   
/bump                                         26  0.013500  0.026242   
/buy                                         149  1.072060  0.762190   
/complete                                     79  0.748924  0.213157   
/initialize                                    2  8.911500  0.615890   
/items/:id.json                             9353  0.003506  0.004846   
/items/edit                                   20  0.004550  0.004605   
/login                                       124  0.178750  0.107716   
/new_items.json                              292  0.013349  0.019295   
/new_items/:id.json                         3784  0.010435  0.017251   
/reports.json                                  2  0.002000  0.000000   
/sell                                        144  0.029042  0.064841   
/settings                                    108  0.044824  0.057166   
/ship                                        121  0.580843  0.365031   
/ship_done                                   129  0.577279  0.364301   
/static/css/main.19393e92.chunk.css            2  0.000500  0.000707   
/static/js/2.ff6e1067.chunk.js                 2  0.001500  0.000707   
/static/js/main.babc3d4d.chunk.js              2  0.004000  0.004243   
/static/js/runtime~main.a8a9905a.js            2  0.000500  0.000707   
/transaction/:id.png                         115  0.001365  0.000985   
/upload/:id.jpg                              116  0.031914  0.058222   
/users/:id.json                             1024  1.157563  1.775952   

                                                             
                                          sum    min    max  
path                                                         
/bump                                   0.351  0.005  0.112  
/buy                                  159.737  0.000  1.638  
/complete                              59.165  0.007  0.846  
/initialize                            17.823  8.476  9.347  
/items/:id.json                        32.788  0.001  0.123  
/items/edit                             0.091  0.001  0.017  
/login                                 22.165  0.063  0.506  
/new_items.json                         3.898  0.001  0.135  
/new_items/:id.json                    39.485  0.002  0.254  
/reports.json                           0.004  0.002  0.002  
/sell                                   4.182  0.000  0.439  
/settings                               4.841  0.000  0.230  
/ship                                  70.282  0.000  0.823  
/ship_done                             74.469  0.000  0.820  
/static/css/main.19393e92.chunk.css     0.001  0.000  0.001  
/static/js/2.ff6e1067.chunk.js          0.003  0.001  0.002  
/static/js/main.babc3d4d.chunk.js       0.008  0.001  0.007  
/static/js/runtime~main.a8a9905a.js     0.001  0.000  0.001  
/transaction/:id.png                    0.157  0.000  0.007  
/upload/:id.jpg                         3.702  0.000  0.262  
/users/:id.json                      1185.345  0.000  8.029

In [106]:
aggregated_df.sort_values(("request_time", "sum"), ascending=False)

request_time                      \
                                           count      mean       std   
path                                                                   
/users/:id.json                             1024  1.157563  1.775952   
/buy                                         149  1.072060  0.762190   
/ship_done                                   129  0.577279  0.364301   
/ship                                        121  0.580843  0.365031   
/complete                                     79  0.748924  0.213157   
/new_items/:id.json                         3784  0.010435  0.017251   
/items/:id.json                             9353  0.003506  0.004846   
/login                                       124  0.178750  0.107716   
/initialize                                    2  8.911500  0.615890   
/settings                                    108  0.044824  0.057166   
/sell                                        144  0.029042  0.064841   
/new_items.json                              292  0.013349  0.019295   
/upload/:id.jpg                              116  0.031914  0.058222   
/bump                                         26  0.013500  0.026242   
/transaction/:id.png                         115  0.001365  0.000985   
/items/edit                                   20  0.004550  0.004605   
/static/js/main.babc3d4d.chunk.js              2  0.004000  0.004243   
/reports.json                                  2  0.002000  0.000000   
/static/js/2.ff6e1067.chunk.js                 2  0.001500  0.000707   
/static/css/main.19393e92.chunk.css            2  0.000500  0.000707   
/static/js/runtime~main.a8a9905a.js            2  0.000500  0.000707   

                                                             
                                          sum    min    max  
path                                                         
/users/:id.json                      1185.345  0.000  8.029  
/buy                                  159.737  0.000  1.638  
/ship_done                             74.469  0.000  0.820  
/ship                                  70.282  0.000  0.823  
/complete                              59.165  0.007  0.846  
/new_items/:id.json                    39.485  0.002  0.254  
/items/:id.json                        32.788  0.001  0.123  
/login                                 22.165  0.063  0.506  
/initialize                            17.823  8.476  9.347  
/settings                               4.841  0.000  0.230  
/sell                                   4.182  0.000  0.439  
/new_items.json                         3.898  0.001  0.135  
/upload/:id.jpg                         3.702  0.000  0.262  
/bump                                   0.351  0.005  0.112  
/transaction/:id.png                    0.157  0.000  0.007  
/items/edit                             0.091  0.001  0.017  
/static/js/main.babc3d4d.chunk.js       0.008  0.001  0.007  
/reports.json                           0.004  0.002  0.002  
/static/js/2.ff6e1067.chunk.js          0.003  0.001  0.002  
/static/css/main.19393e92.chunk.css     0.001  0.000  0.001  
/static/js/runtime~main.a8a9905a.js     0.001  0.000  0.001